## Make_cfos_count_df
The purpose of this notebook is to create the "NxK" dataframe, i.e. a CSV file where rows are brains and columns are the raw cell counts in each PMA brain region. 

In [4]:
import os, glob
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import edt
import seaborn as sns

%matplotlib inline

In [15]:
basepath = '/jukebox/wang/Jess/lightsheet_output'
batches = ['201810_adultacutePC_ymaze_cfos',
           '201904_ymaze_cfos',
           '201908_tpham_ymaze_cfos',
           '202002_cfos',
           '202010_cfos']

In [17]:
for batch in batches:
    batch_dir = os.path.join(basepath,batch,'processed')
    animal_dirs = glob.glob(batch_dir+'/*/')
    for d in animal_dirs:
        cells_transformed_file = os.path.join(d,'clearmap_cluster_output/cells_transformed_to_Atlas.npy')
        if not os.path.exists(cells_transformed_file):
            cells_transformed_file = os.path.join(d,'ClearMapClusterOutput/cells_transformed_to_Atlas.npy')
        print(os.path.exists(cells_transformed_file))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
